In [1]:
%matplotlib inline

from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [2]:
RANDOM_SEED = 42
MAX_TOKEN_COUNT = 512
BERT_MODEL_NAME = 'bert-base-cased'
TEXT_COLUMN_NAME = 'text'
LABEL_COLUMN_NAME = 'hs'

In [3]:
df = pd.read_csv('../data/all-processed/B_english_basile_processed.csv')

In [4]:
empty_real_index = [index for index, text in enumerate(df.text.tolist()) if str(text).strip()==""] 
df.iloc[empty_real_index]

,text,hs


In [5]:
df

,text,hs
0,"hurray , saving us $ $ $ many ways @ potus @ r...",1
1,would young fighting age men vast majority one...,1
2,@ kamalaharris illegals dump kids border like ...,1
3,ny times : 'nearly white ' states pose 'an arr...,0
4,orban brussels : european leaders ignoring peo...,0
...,...,...
12995,@ samenvers unfollowed ? fuck pussy,0
12996,@ danreynolds stfu bitch ! go make satanic mus...,1
12997,"@ 2beornotbeing honey , fellow white chick , l...",0
12998,"hate bitches talk niggaz kids , everybody cant...",1


In [6]:
df[LABEL_COLUMN_NAME].value_counts()

0    7530
1    5470
Name: hs, dtype: int64

In [7]:
df['text'].map(
    lambda text: len(str(text).split(" "))
).describe()

count    13000.000000
mean        18.853385
std         10.067602
min          1.000000
25%         11.000000
50%         17.000000
75%         24.000000
max        121.000000
Name: text, dtype: float64

In [8]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df.shape, val_df.shape

((12350, 2), (650, 2))

In [9]:
BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [ ]:
sample_row = df.iloc[0]
sample_comment = sample_row[TEXT_COLUMN_NAME]
sample_labels = sample_row[LABEL_COLUMN_NAME]

print(sample_comment)

In [ ]:
class HateDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = MAX_TOKEN_COUNT
    ):
    
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        text = data_row[TEXT_COLUMN_NAME]
        labels = data_row[LABEL_COLUMN_NAME]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return dict(
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            labels=labels
        )   

In [ ]:
train_dataset = HateDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

sample_item = train_dataset[0]
sample_item.keys()

In [ ]:
sample_item["labels"]

In [ ]:
sample_item["input_ids"]

In [ ]:
sample_item["attention_mask"]

In [ ]:
class HateDataModule(pl.LightningDataModule):
    def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        
    def setup(self, stage=None):
        self.train_dataset = HateDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = HateDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size
        )

In [ ]:
N_EPOCHS = 10
BATCH_SIZE = 16

data_module = HateDataModule(
  train_df,
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [ ]:
import pytorch_lightning as pl
import torchmetrics

class HateTagger(pl.LightningModule):
    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.i2h = nn.Linear(self.bert.config.hidden_size, 128)
        self.h2o = nn.Linear(128, 1)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()
        self.auroc = torchmetrics.AUROC()
        self.accuracy = torchmetrics.Accuracy()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.i2h(output.pooler_output)
        output = self.h2o(output)
        output = torch.sigmoid(output).flatten()
        loss = 0
        if labels is not None:
            loss = self.criterion(output.to(dtype=float), labels.to(dtype=float))
        return loss, output

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        # self.log("train_AUROC", self.auroc(outputs, labels), prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        self.log("train_acc", self.calculate_accuracy(outputs, labels), prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        return {"loss": loss, "predictions": outputs, "labels": labels}
        
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        # self.log("val_AUROC", self.auroc(outputs, labels), prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        self.log("val_acc", self.calculate_accuracy(outputs, labels), prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        # self.log("test_AUROC", self.auroc(outputs, labels), prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        self.log("test_acc", self.calculate_accuracy(outputs, labels), prog_bar=True, logger=True, batch_size=BATCH_SIZE)
        return loss

    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        roc_auc = self.auroc(predictions, labels)
        self.logger.experiment.add_scalar("roc_auc/Train", roc_auc, self.current_epoch)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )
    
    def calculate_auroc(self, outputs, labels):
        return self.auroc(torch.round(outputs).to(int), labels.to(int))

    def calculate_accuracy(self, outputs, labels):
        return self.accuracy(torch.round(outputs).to(int), labels.to(int))

In [ ]:
model = HateTagger(
  n_warmup_steps=20,
  n_training_steps=100
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="fake-news-ff")

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
trainer = pl.Trainer(
  logger=logger,
  checkpoint_callback=checkpoint_callback,
  callbacks=[early_stopping_callback],
  max_epochs=10,
  gpus=1,
  progress_bar_refresh_rate=30
)

In [ ]:
trainer.fit(model, data_module)
# trainer.fit(model, data_loader1)

In [ ]:
# checkpoint = torch.load('lightning_logs/fake-news-snopes/version_40/checkpoints/epoch=2-step=7886.ckpt')
# model.load_from_checkpoint('lightning_logs/fake-news-snopes/version_40/checkpoints/epoch=2-step=7886.ckpt')
trainer.test(model, data_module)